In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from keras.preprocessing import image
import serial
import csv

# Load the model
MODEL = tf.keras.models.load_model(r"C:\Users\jacki\Downloads\My_Dataset\model.h5")

# Define the class labels (example labels, replace with your actual labels)
arr1 = ['pothole', 'normal']  # Replace with your actual class labels

# Initialize video capture
vid = cv2.VideoCapture(2)  # Try with the correct index for your setup

if not vid.isOpened():
    print("Error: Unable to open video capture.")
else:
    print("Video capture opened successfully.")

# Initialize serial communication with Arduino for GPS data
try:
    arduino = serial.Serial('COM4', 9600, timeout=1)  # Adjust COM port as necessary
    print("Serial port opened successfully.")
except serial.SerialException as e:
    print(f"Error: {e}")
    arduino = None

# Open CSV file for writing GPS data
csvfile = open('gps_data4.csv', 'w', newline='')
csv_writer = csv.writer(csvfile)
csv_writer.writerow(['Latitude', 'Longitude'])  # Write header

saved_coordinates = None
latitude = None
longitude = None

while True:
    success, img = vid.read()

    if not success:
        print("Error: Unable to capture frame.")
        break

    # Preprocess the image
    img1 = img / 255.0  # Normalize the image
    img1 = cv2.resize(img1, (200, 200), interpolation=cv2.INTER_NEAREST)
    X = image.img_to_array(img1)
    X = np.expand_dims(X, axis=0)
    images = np.vstack([X])

    # Make prediction
    prediction = MODEL.predict(images)

    # Find the class with the highest probability
    index = np.argmax(prediction)
    max_prob = prediction[0][index]
    detected_class = arr1[index]  # Use the correct class label

    # Display the result on the frame
    text = f"{detected_class} is detected with {max_prob * 100:.2f}% confidence"
    img = cv2.putText(img, text, (5, 60), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0, 0, 255), 1)

    # Only read GPS data and store it if a pothole is detected
    if detected_class == 'pothole' and arduino is not None and arduino.is_open:
        try:
            gps_data = arduino.readline().decode('utf-8').strip()
            print(f"Received GPS data: {gps_data}")

            # Parse GPS data
            if gps_data.startswith("Latitude:"):
                try:
                    latitude = float(gps_data.split("Latitude:")[1].strip())
                    print(f"Parsed latitude: {latitude}")
                except ValueError:
                    print(f"Invalid latitude data: {gps_data}")
                    latitude = None
            elif gps_data.startswith("Longitude:"):
                try:
                    longitude = float(gps_data.split("Longitude:")[1].strip())
                    print(f"Parsed longitude: {longitude}")
                except ValueError:
                    print(f"Invalid longitude data: {gps_data}")
                    longitude = None

            # If both latitude and longitude have been received
            if latitude is not None and longitude is not None:
                coordinates = f"{latitude}, {longitude}"
                print(f"Coordinates to save: {coordinates}")

                if saved_coordinates is None or coordinates != saved_coordinates:
                    csv_writer.writerow([latitude, longitude])
                    print(f"New GPS coordinates saved: {coordinates}")
                    saved_coordinates = coordinates

                # Reset latitude and longitude for the next reading
                latitude = None
                longitude = None
        except serial.SerialException as e:
            print(f"Error reading from serial port: {e}")

    # Display the frame
    cv2.imshow("Video", img)
    
    # Check for 'ESC' key press to exit the loop
    if cv2.waitKey(1) & 0xFF == 27:
        break

# Release resources
vid.release()
csvfile.close()
if arduino is not None and arduino.is_open:
    arduino.close()
cv2.destroyAllWindows()


    

Video capture opened successfully.
Serial port opened successfully.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Received GPS data: 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Received GPS data: GPS Receiver Initialized
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Received GPS data: Latitude: 26.476945
Parsed latitude: 26.476945
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Received GPS data: Longitude: 90.299201
Parsed longitude: 90.299201
Coordinates to save: 26.476945, 90.299201
New GPS coordinates saved: 26.476945, 90.299201
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Received GPS data: Latitude: 26.476945
Parsed latitude: 26.476945
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Received GPS data: Longitude: 90.299201
Parsed longitude: 90.299201
Coordinates to save: 26.476945, 90.299201
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Received GPS data: Latitude: 26.476945
Parsed latitude: 26.476945
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Received GPS data: Longitude: 90.299201
Parsed longitude: 90

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Received GPS data: Latitude: 26.476938
Parsed latitude: 26.476938
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Received GPS data: Longitude: 90.299217
Parsed longitude: 90.299217
Coordinates to save: 26.476938, 90.299217
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Received GPS data: Latitude: 26.476938
Parsed latitude: 26.476938
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Received GPS data: Longitude: 90.299217
Parsed longitude: 90.299217
Coordinates to save: 26.476938, 90.299217
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Received GPS data: Latitude: 26.476938
Parsed latitude: 26.476938
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Received GPS data: Longitude: 90.299217
Parsed longitude: 90.299217
Coordinates to save: 26.476938, 90.299217
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Received GPS data: Latitude: 26.476938
Parsed latitude: 26.476938
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Received GPS data: Longitude: 90.299217
Parsed longitude: 90.299217
Coordinates to save: 26.476938, 90

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Received GPS data: Longitude: 90.299217
Parsed longitude: 90.299217
Coordinates to save: 26.47694, 90.299217
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Received GPS data: Latitude: 26.476940
Parsed latitude: 26.47694
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Received GPS data: Longitude: 90.299217
Parsed longitude: 90.299217
Coordinates to save: 26.47694, 90.299217
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Received GPS data: Latitude: 26.476940
Parsed latitude: 26.47694
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Received GPS data: Longitude: 90.299217
Parsed longitude: 90.299217
Coordinates to save: 26.47694, 90.299217
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Received GPS data: Latitude: 26.476940
Parsed latitude: 26.47694
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Received GPS data: Longitude: 90.299217
Parsed longitude: 90.299217
Coordinates to save: 26.47694, 90.299217
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Received GPS data: Latitude: 26.476940
Parsed latitude: 26.47694


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Received GPS data: Latitude: 26.476943
Parsed latitude: 26.476943
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Received GPS data: Longitude: 90.299209
Parsed longitude: 90.299209
Coordinates to save: 26.476943, 90.299209
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Received GPS data: Latitude: 26.476943
Parsed latitude: 26.476943
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Received GPS data: Longitude: 90.299209
Parsed longitude: 90.299209
Coordinates to save: 26.476943, 90.299209
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Received GPS data: Latitude: 26.476943
Parsed latitude: 26.476943
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Received GPS data: Longitude: 90.299209
Parsed longitude: 90.299209
Coordinates to save: 26.476943, 90.299209
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Received GPS data: Latitude: 26.476943
Parsed latitude: 26.476943
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Received GPS data: Longitude: 90.299209
Parsed longitude: 90.299209
Coordinates to save: 26.476943, 90

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Received GPS data: Longitude: 90.299186
Parsed longitude: 90.299186
Coordinates to save: 26.476938, 90.299186
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Received GPS data: Latitude: 26.476938
Parsed latitude: 26.476938
1/1 ━━━━━━━━━━━━━━━━━━━━ 0

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Received GPS data: Latitude: 26.476942
Parsed latitude: 26.476942
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Received GPS data: Longitude: 90.299179
Parsed longitude: 90.299179
Coordinates to save: 26.476942, 90.299179
New GPS coordinates saved: 26.476942, 90.299179
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Received GPS data: Latitude: 26.476942
Parsed latitude: 26.476942
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━